<a href="https://colab.research.google.com/github/manishiitg/ML_Experments/blob/master/keras/deep_learning_word2vec_model_with_CNN%2C_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying out CNN and LSTM models for text classification. 

Previously i tried out simple FF network, but the results where not very good. So exprimenting with CNN/LSTM models 

In [0]:
# https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

%matplotlib inline

import pandas as pd
import numpy as np

import spacy 
nlp = spacy.load("en_core_web_sm")

from sklearn.datasets import fetch_20newsgroups

from gensim.models import KeyedVectors

from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;

from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
import matplotlib.pyplot as plt;
from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import TfidfVectorizer


from collections import defaultdict


In [0]:
def normalize(comment, lowercase=True, remove_stopwords=True):
    if lowercase:
        comment = comment.lower()
    lines = comment.splitlines()
    lines = [x.strip(' ') for x in lines]
    lines = [x.replace('"', '') for x in lines]
    lines = [x.replace('\\"', '') for x in lines]
    lines = [x.replace(u'\xa0', u'') for x in lines]
    comment = " ".join(lines)
    doc = nlp(comment)

    # for token in doc:
    #   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #     token.shape_, token.is_alpha, token.is_stop)

    words = [token for token in doc if token.is_stop !=
             True and token.is_punct != True]
    # return " ".join(words)
    lemmatized = list()
    for word in words:
        lemma = word.lemma_.strip()
        if lemma:
            lemmatized.append(lemma)
    return lemmatized

In [0]:
import os

def writetofile(dir, filename, data):
    if not os.path.exists(dir):
        os.makedirs(dir)
    f = os.path.join(dir,str(filename))

    with open(f, 'wb') as the_file:
      the_file.write(data)


categories = [ 'comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','comp.windows.x']
news = fetch_20newsgroups(subset="train", categories=categories)


clean_data  = []

# print(news.keys())

# print(news["filenames"][:10])

# print(len(news["data"][:1000]))

# print(news["target_names"][:50])
# print(news["target"][:10])


max_limit = 4000

targets = news["target"][:max_limit]
filenames = news["filenames"][:max_limit]
news = news["data"][:max_limit]

dir = "news_group_cleaned"

print("cleaning data")
for i, row in enumerate(news):
  filename = filenames[i]
  filename = filename[(filename.rfind('/'))+1:]
  if os.path.exists(filename):
    with open(os.path.join(dir, filename), 'r') as content_file:
      data = content_file.read()
      cleaned = data.split(" ")
  else:
    cleaned = normalize(row)
    writetofile(dir, filename, " ".join(cleaned).encode("utf-8"))
    
  
  clean_data.append(cleaned)

print("data cleaned")

# print(clean_data[10])

cleaning data
data cleaned


In [0]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.Word2Vec(clean_data)


# model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

#https://stackoverflow.com/questions/45159693/word2vec-models-consist-of-characters-instead-of-words

2019-10-14 20:32:13,548 : INFO : collecting all words and their counts
2019-10-14 20:32:13,550 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-14 20:32:13,718 : INFO : collected 66979 word types from a corpus of 470869 raw words and 2936 sentences
2019-10-14 20:32:13,719 : INFO : Loading a fresh vocabulary
2019-10-14 20:32:13,777 : INFO : effective_min_count=5 retains 7758 unique words (11% of original 66979, drops 59221)
2019-10-14 20:32:13,778 : INFO : effective_min_count=5 leaves 392121 word corpus (83% of original 470869, drops 78748)
2019-10-14 20:32:13,814 : INFO : deleting the raw counts dictionary of 66979 items
2019-10-14 20:32:13,817 : INFO : sample=0.001 downsamples 33 most-common words
2019-10-14 20:32:13,818 : INFO : downsampling leaves estimated 326228 word corpus (83.2% of prior 392121)
2019-10-14 20:32:13,853 : INFO : estimated required memory for 7758 words and 100 dimensions: 10085400 bytes
2019-10-14 20:32:13,855 : INFO : resetting 

In [0]:
#prepare text

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2


tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_data)
sequences = tokenizer.texts_to_sequences(clean_data)

word_index = tokenizer.word_index


print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(targets))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


Using TensorFlow backend.
/mnt/c/work/testflask/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/c/work/testflask/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/c/work/testflask/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/c/work/testflask/env/lib/python3.6/site-pa

Found 66979 unique tokens.
Shape of data tensor: (2936, 1000)
Shape of label tensor: (2936, 5)


In [0]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

# this is index of words which we tokenized i.e a map with word ==> index

for word, i in word_index.items():
  if word in model.wv.vocab:
    # print("found word" , word)
    embedding_vector = model.wv[word]
    embedding_matrix[i] = embedding_vector
  # else:
  #   print("work not found" , word)
  # if i> 100:
  #   break
  # else:
  #   print("word not found", word)

In [0]:


from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten

model = Sequential()

model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(categories), activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()


model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=12, batch_size=128,shuffle=True)

# scores = model.evaluate(X_test, Y_test, verbose=1)

# print("Accuracy: ", scores[1])
# print("Loss: ", scores[0])


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 1000, 100)         6698000   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 199, 128)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 39, 128)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4992)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)             

In [0]:
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten
from keras.layers import LSTM


model = Sequential()

model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(categories), activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()


model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=12, batch_size=32,shuffle=True)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1000, 100)         6698000   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 199, 128)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_21 (Dense)             (None, 5)                 645       
Total params: 6,910,869
Trainable params: 212,869
Non-trainable params: 6,698,000
_____________________________________